In [1]:
import requests
from bs4 import BeautifulSoup as BS
import pandas as pd
import re
import numpy as np
import json

jan 18 - march 13

**Scraping the web to pull a table out that contains champion stats by patch**

In [2]:
patches = [13.1, 13.3, 13.4, 13.5, 13.6]

In [3]:
URL = 'https://www.metasrc.com/5v5/br/{patches}/stats?ranks=diamond'.format(patches = 13.1)

response = requests.get(URL)

soup = BS(response.text)

In [4]:
df = []

for patch in patches:
    URL = 'https://www.metasrc.com/5v5/br/{patches}/stats?ranks=diamond'.format(patches = patch)

    response = requests.get(URL)

    soup = BS(response.text)
    
    temp = pd.read_html(str(soup.findAll('table', attrs={'class' : 'stats-table'})))[0]
    temp['patch'] = patch
    df.append(temp)

In [5]:
patchInfo = (
    pd.concat(df)
    .reset_index(drop = True)[['Name', 'Role', 'Score', 'Win %', 'Pick %', 'patch']]
    .rename(columns = {
        'Name' : 'championName',
        'Role' : 'lane',
        'Score' : 'score',
        'Win %' : 'winPercent',
        'Pick %' : 'pickPercent'
    })
)

patchInfo['winPercent'] = patchInfo['winPercent'].str.rstrip("%").astype(float)/100
patchInfo['pickPercent'] = patchInfo['pickPercent'].str.rstrip("%").astype(float)/100

In [6]:
champion = []
for i in range(0, len(patchInfo['championName'])):
    
    champion.append(patchInfo['championName'][i][0:int(len(patchInfo['championName'][i])/2)])

patchInfo['championName'] = champion

patchInfo['championName'] = patchInfo['championName'].str.lower()
patchInfo['championName'] = patchInfo['championName'].str.replace(' ', '')
patchInfo['championName'] = patchInfo['championName'].str.replace("'", '')
patchInfo['championName'] = patchInfo['championName'].str.replace(".", '')

C:\Users\hcgre\AppData\Local\Temp\ipykernel_9240\1636505631.py:11: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  patchInfo['championName'] = patchInfo['championName'].str.replace(".", '')


In [7]:
patchInfo['lane'] = (
    np.where(patchInfo['lane'] == 'MID', 'MIDDLE',
    np.where(patchInfo['lane'] == 'SUPPORT', 'UTILITY',
    np.where(patchInfo['lane'] == 'TOP', 'TOP',
    np.where(patchInfo['lane'] == 'ADC', 'BOTTOM', 'JUNGLE'
            ))))
)

In [8]:
patchInfo = patchInfo.rename(columns = {'lane' : 'teamPosition'})

**Using the date of the match to apply which patch the game was played on**

In [9]:
matchDates = pd.read_csv('../data/matchdates.csv')

matchDates['patch'] = (
    np.where((matchDates['matchcreationDate'] >= '2023-01-11') & (matchDates['matchcreationDate'] < '2023-02-09'), 13.1, 
    np.where((matchDates['matchcreationDate'] >= '2023-02-09') & (matchDates['matchcreationDate'] < '2023-02-23'), 13.3,
    np.where((matchDates['matchcreationDate'] >= '2023-02-23') & (matchDates['matchcreationDate'] < '2023-03-08'), 13.4,
    np.where((matchDates['matchcreationDate'] >= '2023-03-08') & (matchDates['matchcreationDate'] < '2023-03-22'), 13.5, 13.6
    ))))
)

matchPatch = matchDates[['matchId', 'patch']]

**Merging lane data and match data with patch data**

In [10]:
positions = pd.read_csv('../data/teamPositions.csv')

positionsTrain = positions.loc[positions['matchId'] <= 7999].reset_index(drop=True)

In [11]:
patchTrain = pd.read_csv('../data/participants_train.csv')[['matchId', 'teamId', 'participantId', 'championName']].merge(
    right = positionsTrain,
    how = 'left',
    on = ['matchId', 'participantId']
).merge(
    right = matchPatch,
    how = 'left',
    on = ['matchId']
)

**Correcting where team position is nan**

In [12]:
naTeamPositions = patchTrain.loc[patchTrain['teamPosition'].isna()].copy()

naTeamPositions['teamPosition'] = np.where(
    (naTeamPositions['participantId'] == 1) | (naTeamPositions['participantId'] == 6), 'TOP',
np.where(
    (naTeamPositions['participantId'] == 2) | (naTeamPositions['participantId'] == 7), 'JUNGLE',
np.where(
    (naTeamPositions['participantId'] == 3) | (naTeamPositions['participantId'] == 8), 'MIDDLE',
np.where(
    (naTeamPositions['participantId'] == 4) | (naTeamPositions['participantId'] == 9), 'BOTTOM', 'UTILITY'
))))

In [13]:
patchTrain = patchTrain.loc[~patchTrain['teamPosition'].isna()]

patchTrain = pd.concat([patchTrain, naTeamPositions]).sort_values(by = ['matchId', 'participantId'])

patchTrain['championName'] = patchTrain['championName'].str.lower()

**Changing champion name 'monkeyking' to 'wukong'**

In [14]:
monkeyKing = patchTrain.loc[patchTrain['championName'] == 'monkeyking'].copy()

monkeyKing['championName'] = 'wukong'

patchTrain = patchTrain.loc[patchTrain['championName'] != 'monkeyking']

patchTrain = pd.concat([patchTrain, monkeyKing]).sort_values(by = ['matchId', 'participantId'])

**Changing champion name 'renata' to 'renataglasc'**

In [15]:
renata = patchTrain.loc[patchTrain['championName'] == 'renata'].copy()

renata['championName'] = 'renataglasc'

patchTrain = patchTrain.loc[patchTrain['championName'] != 'renata']

patchTrain = pd.concat([patchTrain, renata]).sort_values(by = ['matchId', 'participantId'])

**Merging champion stats by patch with match data**

In [16]:
patchTrain = patchTrain.merge(
    right = patchInfo,
    how = 'left',
    on = ['championName', 'teamPosition', 'patch']
)

In [17]:
patchTrain.isna().sum()

matchId             0
teamId              0
participantId       0
championName        0
teamPosition        0
patch               0
score            3331
winPercent       3331
pickPercent      3331
dtype: int64

**Dealing with the large amount of nan**
* unusalPositions - there was no current patch data for the champion/position combo but there was past data which was used to subtitute.
* potenialLaneSwap - game placed player in unfavorable position and another player on the team swapped positions with them.

In [18]:
unusalPositions = (
    patchTrain.loc[patchTrain['winPercent'].isna()][['matchId', 'teamId', 'participantId', 'championName', 'teamPosition']]
    .merge(
        right = patchInfo[['championName', 'teamPosition', 'patch', 'score', 'winPercent', 'pickPercent']],
        how = 'left',
        on = ['championName', 'teamPosition']
    ).sort_values(by = 'pickPercent', ascending = False)
    .drop_duplicates(subset = ['matchId', 'participantId'])
)

stillNan = unusalPositions.loc[unusalPositions['winPercent'].isna()]

unusalPositions = unusalPositions.loc[~unusalPositions['winPercent'].isna()]

In [19]:
potenialLaneSwap = (
    stillNan[['matchId', 'teamId', 'participantId', 'championName', 'teamPosition']].merge(
        right = patchInfo[['championName', 'patch', 'score', 'winPercent', 'pickPercent']],
        how = 'left',
        on = ['championName']
    ).sort_values(by = 'pickPercent', ascending = False)
    .drop_duplicates(subset = ['matchId', 'participantId'])
)

potenialLaneSwap = potenialLaneSwap.loc[~potenialLaneSwap['winPercent'].isna()]

In [20]:
patchTrain = patchTrain.loc[~patchTrain['winPercent'].isna()]

patchTrain = pd.concat([patchTrain, unusalPositions, potenialLaneSwap]).sort_values(by = ['matchId', 'participantId'])

patchTrain.isna().sum()

matchId          0
teamId           0
participantId    0
championName     0
teamPosition     0
patch            0
score            0
winPercent       0
pickPercent      0
dtype: int64

**Collapsing the 10 rows per game to 1 by grouping by match id and taking the average of score and win percentage**

In [21]:
patchTrain = (
    patchTrain[['matchId', 'teamId', 'score', 'winPercent']]
    .groupby(by = ['matchId', 'teamId'])
    .mean()
    .reset_index()
)

In [22]:
patchTrain = (
    patchTrain.loc[patchTrain['teamId'] == 100][['matchId', 'score', 'winPercent']]
    .rename(columns = {
        'score' : 'blueChampScore',
        'winPercent' : 'blueChampWinPercent'
    })
).merge(
    right = (patchTrain.loc[patchTrain['teamId'] == 200][['matchId', 'score', 'winPercent']]
    .rename(columns = {
        'score' : 'redChampScore',
        'winPercent' : 'redChampWinPercent'
    })),
    how = 'left',
    on = ['matchId']
)

In [23]:
#patchTrain.to_csv('../data/patchTrain.csv', index = False)

**Repeating the above steps for test data**

**Merging lane data and match data with patch data**

In [24]:
positionsTest = positions.loc[positions['matchId'] >= 8000].reset_index(drop=True)

In [25]:
patchTest = pd.read_csv('../data/participants_test.csv')[['matchId', 'teamId', 'participantId', 'championName']].merge(
    right = positionsTest,
    how = 'left',
    on = ['matchId', 'participantId']
).merge(
    right = matchPatch,
    how = 'left',
    on = ['matchId']
)

**Correcting where team position is nan**

In [26]:
naTeamPositions = patchTest.loc[patchTest['teamPosition'].isna()].copy()

naTeamPositions['teamPosition'] = np.where(
    (naTeamPositions['participantId'] == 1) | (naTeamPositions['participantId'] == 6), 'TOP',
np.where(
    (naTeamPositions['participantId'] == 2) | (naTeamPositions['participantId'] == 7), 'JUNGLE',
np.where(
    (naTeamPositions['participantId'] == 3) | (naTeamPositions['participantId'] == 8), 'MIDDLE',
np.where(
    (naTeamPositions['participantId'] == 4) | (naTeamPositions['participantId'] == 9), 'BOTTOM', 'UTILITY'
))))

In [27]:
patchTest = patchTest.loc[~patchTest['teamPosition'].isna()]

patchTest = pd.concat([patchTest, naTeamPositions]).sort_values(by = ['matchId', 'participantId'])

patchTest['championName'] = patchTest['championName'].str.lower()

**Changing champion name 'monkeyking' to 'wukong'**

In [28]:
monkeyKing = patchTest.loc[patchTest['championName'] == 'monkeyking'].copy()

monkeyKing['championName'] = 'wukong'

patchTest = patchTest.loc[patchTest['championName'] != 'monkeyking']

patchTest = pd.concat([patchTest, monkeyKing]).sort_values(by = ['matchId', 'participantId'])

**Changing champion name 'renata' to 'renataglasc'**

In [29]:
renata = patchTest.loc[patchTest['championName'] == 'renata'].copy()

renata['championName'] = 'renataglasc'

patchTest = patchTest.loc[patchTest['championName'] != 'renata']

patchTest = pd.concat([patchTest, renata]).sort_values(by = ['matchId', 'participantId'])

**Merging champion stats by patch with match data**

In [30]:
patchTest = patchTest.merge(
    right = patchInfo,
    how = 'left',
    on = ['championName', 'teamPosition', 'patch']
)

In [31]:
patchTest.isna().sum()

matchId            0
teamId             0
participantId      0
championName       0
teamPosition       0
patch              0
score            853
winPercent       853
pickPercent      853
dtype: int64

**Dealing with the large amount of nan**
* unusalPositions - there was no current patch data for the champion/position combo but there was past data which was used to subtitute.
* potenialLaneSwap - game placed player in unfavorable position and another player on the team swapped positions with them.

In [32]:
unusalPositions = (
    patchTest.loc[patchTest['winPercent'].isna()][['matchId', 'teamId', 'participantId', 'championName', 'teamPosition']]
    .merge(
        right = patchInfo[['championName', 'teamPosition', 'patch', 'score', 'winPercent', 'pickPercent']],
        how = 'left',
        on = ['championName', 'teamPosition']
    ).sort_values(by = 'pickPercent', ascending = False)
    .drop_duplicates(subset = ['matchId', 'participantId'])
)

stillNan = unusalPositions.loc[unusalPositions['winPercent'].isna()]

unusalPositions = unusalPositions.loc[~unusalPositions['winPercent'].isna()]

In [33]:
potenialLaneSwap = (
    stillNan[['matchId', 'teamId', 'participantId', 'championName', 'teamPosition']].merge(
        right = patchInfo[['championName', 'patch', 'score', 'winPercent', 'pickPercent']],
        how = 'left',
        on = ['championName']
    ).sort_values(by = 'pickPercent', ascending = False)
    .drop_duplicates(subset = ['matchId', 'participantId'])
)

potenialLaneSwap = potenialLaneSwap.loc[~potenialLaneSwap['winPercent'].isna()]

In [34]:
patchTest = patchTest.loc[~patchTest['winPercent'].isna()]

patchTest = pd.concat([patchTest, unusalPositions, potenialLaneSwap]).sort_values(by = ['matchId', 'participantId'])

patchTest.isna().sum()

matchId          0
teamId           0
participantId    0
championName     0
teamPosition     0
patch            0
score            0
winPercent       0
pickPercent      0
dtype: int64

**Collapsing the 10 rows per game to 1 by grouping by match id and taking the average of score and win percentage**

In [35]:
patchTest = (
    patchTest[['matchId', 'teamId', 'score', 'winPercent']]
    .groupby(by = ['matchId', 'teamId'])
    .mean()
    .reset_index()
)

In [36]:
patchTest = (
    patchTest.loc[patchTest['teamId'] == 100][['matchId', 'score', 'winPercent']]
    .rename(columns = {
        'score' : 'blueChampScore',
        'winPercent' : 'blueChampWinPercent'
    })
).merge(
    right = (patchTest.loc[patchTest['teamId'] == 200][['matchId', 'score', 'winPercent']]
    .rename(columns = {
        'score' : 'redChampScore',
        'winPercent' : 'redChampWinPercent'
    })),
    how = 'left',
    on = ['matchId']
)

In [38]:
#patchTest.to_csv('../data/patchTest.csv', index = False)